## 請手刻gradient descent實作logistic regression

In [2]:
import numpy as np

##### read X_train and Y_train

In [20]:
import csv

# 106 columns

X_train = []
Y_train = []

with open('data/X_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        X_train.append([float(i) for i in row] + [1.0])
        
        '''continuous_idxs = []
        for idx2, num in enumerate(row):
            if float(num) > 1:
                continuous_idxs.append(idx2)
        print(continuous_idxs)'''

X_train = np.array(X_train)
for col in [0,1,3,4,5]: # [0,1,3,4,5] only, shouldn't normalize one-hot columns, will be affected by data
    if np.std(X_train[:,col]) != 0:
        X_train[:,col] = np.divide((X_train[:,col] - np.average(X_train[:,col])), np.std(X_train[:,col]))

# [[data1]
#  [data2]]

with open('data/Y_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        Y_train.append(float(row[0]))

Y_train = np.array(Y_train)
# [y1 y2]


In [40]:
'''print(len(X_train[0]))
# print(X_train[0:2])
# print(X_train[0:3])
print(X_train[7])
print(Y_train[7])'''

'print(len(X_train[0]))\n# print(X_train[0:2])\n# print(X_train[0:3])\nprint(X_train[7])\nprint(Y_train[7])'

##### define sigmoid function for numpy

In [5]:
def sigmoid(x): # this sigmoid works with numpy arrays
    res =  1.0 / (1 + np.exp(-x))
    return np.clip(res, 0.00000000000001, 0.99999999999999)

##### start training, split X_train into X_real_train and X_valid

##### currently not caring about validation set, directly train with X_real_train = X_train

###### accuracy stuck at 0.82

In [36]:
# optimizations to consider when hand-crafting everything: feature scaling (done), l1 l2 norm

# grab validation set
import random
val_idxs = random.sample(range(0, len(X_train)), int(len(X_train) * 0.2))
X_valid = X_train[val_idxs]
Y_valid = Y_train[val_idxs]

X_real_train = np.copy(np.delete(X_train, val_idxs, 0))
Y_real_train = np.copy(np.delete(Y_train, val_idxs, 0))
'''X_real_train = X_train
Y_real_train = Y_train'''

# old batch = 50 #len(X_real_train)
batch = 800 #800 seems promising
lr = 0.01
decay = 0.95
my_lambda = 0 # l2 norm

prev_grads = [] # for adagrad


# initialize weights
weights = np.zeros(len(X_train[0])) # we already appended bias to X_train

#===================saving previous weights======================
# weights = weights_saved

for epoch in range(0,10):
    # randomly decide training order in this epoch
    # [41,6,0,231...]
    random_idxs = random.sample(range(0, len(X_real_train)), len(X_real_train))
    for i in range(0, len(random_idxs), batch):
        X_batch = X_real_train[random_idxs[i: min(i+batch, len(random_idxs))]]
        Y_batch = Y_real_train[random_idxs[i: min(i+batch, len(random_idxs))]]
        Y_pred = np.dot(X_batch, weights)
        Y_pred = sigmoid(Y_pred)
        Y_pred = np.around(Y_pred)

        # get gradients and update weights
        gradient = np.zeros(len(X_train[0]))
        for idx_batch in range(0, len(X_batch)):
            '''if np.isnan(((Y_batch[idx_batch] - Y_pred[idx_batch]) * X_batch[idx_batch] + l2)[0]):
                print('weights', weights)
                print('Y_batch', Y_batch[idx_batch])
                print('Y_pred', Y_pred[idx_batch])
                print('X_batch', X_batch[idx_batch])
                print('l2', l2)'''
            l2 = 2 * my_lambda * np.append(weights[:-1],0)
            # print('gradient adding', - ((Y_batch[idx_batch] - Y_pred[idx_batch]) * X_batch[idx_batch] + l2))
            gradient += - ((Y_batch[idx_batch] - Y_pred[idx_batch]) * X_batch[idx_batch] + l2)
        gradient /= batch
        
        ## prev_grads.append(gradient + 0.00000001) # to avoid dividing by zero using adagrad
        # print('gradient', gradient)
        # print('prev_grads', prev_grads)
        ## adagrad_denom = np.sqrt(sum(np.array(prev_grads) ** 2))
        ## new_grad = gradient / adagrad_denom
        # print('adagrad denom', adagrad_denom)
        # print('new grad', new_grad)
        # print(new_lr)
        weights = weights - lr * gradient * pow(decay, epoch)  # - lr * new_grad
            
    if True:
        train_pred = np.dot(X_real_train, weights)
        train_pred = sigmoid(train_pred)
        train_pred = np.around(train_pred)
        '''print(train_pred[0:20])
        print(Y_real_train[0:20])'''
        print(epoch+1, 'training accuracy', np.sum(train_pred == Y_real_train) / (len(Y_real_train)))

        valid_pred = np.dot(X_valid, weights)
        valid_pred = sigmoid(valid_pred)
        valid_pred = np.around(valid_pred)
        print(epoch+1, 'validation accuracy', np.sum(valid_pred == Y_valid) / (len(Y_valid)))

1 training accuracy 0.827824484625
1 validation accuracy 0.837223587224
2 training accuracy 0.83028139276
2 validation accuracy 0.839987714988
3 training accuracy 0.845291565895
3 validation accuracy 0.854269041769
4 training accuracy 0.840224192867
4 validation accuracy 0.846744471744
5 training accuracy 0.831740181965
5 validation accuracy 0.836916461916
6 training accuracy 0.848285922684
6 validation accuracy 0.858722358722
7 training accuracy 0.835809436063
7 validation accuracy 0.84398034398
8 training accuracy 0.846328074014
8 validation accuracy 0.855958230958
9 training accuracy 0.84678874429
9 validation accuracy 0.854422604423
10 training accuracy 0.846251295635
10 validation accuracy 0.858722358722


In [39]:
'''# 0.82
weights_saved = weights
# print(weights_saved)
np.save('weights_logistic', weights_saved)'''

"# 0.82\nweights_saved = weights\n# print(weights_saved)\nnp.save('weights_logistic', weights_saved)"

In [37]:
X_test = []

with open('data/X_test.csv', 'rt', encoding='big5') as infile:
    reader = csv.reader(infile)

    row1 = next(reader) # skip headings
    for row in reader:
        X_test.append([float(i) for i in row] + [1.0])
            
X_test = np.array(X_test)
for col in [0,1,3,4,5]:
    if np.std(X_test[:,col]) != 0:
        X_test[:,col] = np.divide((X_test[:,col] - np.average(X_test[:,col])), np.std(X_test[:,col]))

pred = np.dot(X_test, weights)
pred = sigmoid(pred)
pred = np.around(pred)
print(pred[0:100])

with open('data/submission.csv', 'wt') as outfile:
    test_writer = csv.writer(outfile)
    test_writer.writerow(['id','label'])
    
    counter = 0
    for num in pred:
        counter += 1
        test_writer.writerow([str(counter),int(num)])
    
            

[ 0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.
  0.  1.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
